# Import Necessary Libraries for all Problems

In [1]:
! pip install selenium 
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import re
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By


import warnings
warnings.filterwarnings('ignore')

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("http://www.amazon.in/")   #for getting amazon platform
driver.maximize_window()
element=input("Searching for :")      #for input method
search=driver.find_elements_by_id("twotabsearchtextbox")  #search field
search[0].send_keys(element)                              #enter keys for searching items
time.sleep(3)
itm=driver.find_elements_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input") #click search button
itm[0].submit()

Searching for :laptop


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
all_urls=[]
pr_names=[]
for i in range(3):
    url=driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for u in url:
        all_urls.append(u.get_attribute('href'))
        
    pr_name=driver.find_elements_by_xpath("//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in pr_name:
        if i.text is None:
            pr_names.append('--')
        else:
            pr_names.append(i.text)
            time.sleep(2)
    driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']").click()
    time.sleep(3)

print(len(all_urls))
print(len(pr_names))

69
69


In [4]:
Brand=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[] 
Availability=[]
for ur in all_urls:
    driver.get(ur)                                                        
    time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-')
    
    try:
        price = driver.find_element_by_xpath("//div[@id='corePriceDisplay_desktop_feature_div']")            # Extracting Price from xpath
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')
        
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return_Exchange.append(ret.text)
    except NoSuchElementException:
        Return_Exchange.append('-')
    try:
        delivry = driver.find_element_by_xpath("//div[id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']")         # Extracting Expected Delivery from xpath
        Expected_Delivery.append(delivry.text)
    except NoSuchElementException:
        Expected_Delivery.append('-')
     
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        Availability.append(avl.text)
    except NoSuchElementException:
        Availability.append('-')
    
        time.sleep(3)
print(len(Brand))
print(len(Price))
print(len(Return_Exchange))
print(len(Expected_Delivery))
print(len(Availability))

69
69
69
69
69


In [5]:
df=pd.DataFrame({"URL":all_urls,"Product names":pr_names,"Brand":Brand,"Price":Price,"Return_Exchange":Return_Exchange,"Expected_Delivery":Expected_Delivery,"Availability":Availability})
df

,URL,Product names,Brand,Price,Return_Exchange,Expected_Delivery,Availability
0,https://www.amazon.in/gp/slredirect/picassoRed...,"Lenovo Ideapad 3 AMD Ryzen 5 5500U 15.6"" FHD T...",Visit the Lenovo Store,"-33%\n₹46,190\n00\nM.R.P.:\n₹68,490.00",7 Days Replacement,-,In stock.
1,https://www.amazon.in/gp/slredirect/picassoRed...,Lenovo IdeaPad Slim 5 11th Gen Intel Core i5 1...,Visit the Lenovo Store,"-42%\n₹58,800\n00\nM.R.P.:\n₹1,01,390.00",7 Days Replacement,-,In stock.
2,https://www.amazon.in/ASUS-15-6-inch-Integrate...,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",Visit the ASUS Store,"-24%\n₹25,990\n00\nM.R.P.:\n₹33,990.00",7 Days Replacement,-,In stock.
3,https://www.amazon.in/Lenovo-35-56cm-Antiglare...,"Lenovo V14 Intel Core i3 11th Gen 14"" (35.56cm...",Visit the Lenovo Store,"-64%\n₹34,999\n00\nM.R.P.:\n₹97,934.00",7 Days Replacement,-,In stock.
4,https://www.amazon.in/Lenovo-IdeaPad-Warranty-...,Lenovo IdeaPad Slim 3 Intel Celeron N4020 15.6...,Visit the Lenovo Store,"-28%\n₹28,980\n00\nM.R.P.:\n₹40,490.00",7 Days Replacement,-,In stock.
...,...,...,...,...,...,...,...
64,https://www.amazon.in/ASUS-VivoBook-i5-1135G7-...,"ASUS VivoBook 14 (2021), Intel Core i5-1135G7 ...",Visit the ASUS Store,-,7 Days Replacement,-,In stock.
65,https://www.amazon.in/Acer-Aspire-Generation-1...,Acer Aspire 3 Intel Core i3 11th Generation 15...,Visit the Acer Store,"-36%\n₹31,790\n00\nM.R.P.:\n₹50,000.00",7 Days Replacement,-,
66,https://www.amazon.in/HP-ProBook-Bluetooth-Int...,(Renewed) HP ProBook 430 G3 6th Gen Intel Core...,Visit the Amazon Renewed Store,-,7 Days Replacement,-,In stock.
67,https://www.amazon.in/gp/slredirect/picassoRed...,Acer Swift X Sfx14-41G Creator Laptop Thin & L...,Visit the Acer Store,-,7 Days Replacement,-,In stock.


In [9]:
#Saving data to csv file
df.to_csv('LAPTOP.csv')

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [10]:
#driver=webdriver.Chrome(r"chromedriver.exe")
#driver.get("https://images.google.com/")
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


delay=10

images=['fruits','cars','Machine Learning','guitars','cakes']     #searching items
links=[]                                                         # scraped url link list

for img in images:
    driver=webdriver.Chrome(r"chromedriver.exe")
    driver.get("https://images.google.com/")
    x=driver.find_element_by_xpath
    f=x("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")   #Sending search items to the search field
    f.send_keys(img)
    time.sleep(3)
    search=x("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")       #Sublit button for searching item
    search.submit()
    time.sleep(5)
    fruit=driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")         #Scraping image src source
    for i in fruit:      
        links.append(i.get_attribute('src'))
        time.sleep(3)
print(len(fruit))  

48


In [11]:
# Scraping first 10 image sources of searched items and making dataframe
all_images=pd.DataFrame([])
all_images["Fruits"]=links[0:10]
all_images["Cars"]=links[48:58]
all_images["Machine Learning"]=links[96:106]
all_images["Guitar"]=links[144:154]
all_images["Cakes"]=links[192:202]
all_images

,Fruits,Cars,Machine Learning,Guitar,Cakes
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


# 4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [12]:
#Python program for scraping mobile details in flipkart platform
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("http://www.flipkart.com/")     #for driver to get flipkart website
time.sleep(3)
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click() #closing login popup dialogue box

search_bar=driver.find_element_by_xpath("//input[@class='_3704LK']")   #Search field to send keys
search_bar.send_keys("one plus nord mobiles")
time.sleep(3)
search_button=driver.find_element_by_xpath("//button[@class='L0Z3Pu']")  #to click search item
search_button.submit()
time.sleep(3)

In [13]:
#scraping elements of modile details
Product_Nam=[]
Br=[]
Price=[]
Product_URL=[]
Battery=[]
Prim_Sec_camera=[]
Display=[]
RAM_ROM=[]
x=driver.find_elements_by_xpath
url=x("//a[@class='_1fQZEK']")   #scraping mobile urls
for i in url:
    Product_URL.append(i.get_attribute('href'))   #href links
time.sleep(3)

#Product names
product=x("//div[@class='_4rR01T']")
for i in product:
    Product_Nam.append(i.text)
#Scraping price 
price=x("//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    Price.append(i.text)
#Scraping RAM_ROM details
ram_rom=x("//ul[@class='_1xgFaf']//li[1]")
for i in ram_rom:
    RAM_ROM.append(str(i.text).replace('|',' '))
#Scraping Display details
dis=x("//ul[@class='_1xgFaf']//li[2]")
for i in dis:
    Display.append(i.text)
#Scraping camera details
prim=x("//ul[@class='_1xgFaf']//li[3]")
for i in prim:
    Prim_Sec_camera.append(i.text)
#Scraping battery details
batt=x("//ul[@class='_1xgFaf']//li[4]")
for i in batt:
    Battery.append(i.text)

print(len(Product_Nam),len(Price),len(Product_URL),len(Battery),len(Display),len(Prim_Sec_camera),len(RAM_ROM))

24 24 24 24 24 24 24


In [14]:
Br=[]    #For product brand names
product=pd.DataFrame(Product_Nam,columns=['product'])
p=product['product'].str.split(' ',expand=True)
Br=p[0]

In [15]:
#Making dataframe
product=pd.DataFrame(Product_Nam,columns=['product'])
p=product['product'].str.split(' ',expand=True)
Mobiles=pd.DataFrame({"Product URL":Product_URL,"Product Name":Product_Nam,"Brand":Br,"Price":Price,
                      "RAM ROM storage":RAM_ROM,"Camera primary&Secondary":Product_Nam,"Battery":Battery,"Display":Display})
Mobiles

,Product URL,Product Name,Brand,Price,RAM ROM storage,Camera primary&Secondary,Battery,Display
0,https://www.flipkart.com/oneplus-nord-ce-2-lit...,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",OnePlus,"₹19,770",6 GB RAM 128 GB ROM,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",5000 mAh Battery,16.74 cm (6.59 inch) Display
1,https://www.flipkart.com/oneplus-nord-ce-2-lit...,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",OnePlus,"₹19,535",6 GB RAM 128 GB ROM,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",5000 mAh Battery,16.74 cm (6.59 inch) Display
2,https://www.flipkart.com/oneplus-nord-2-5g-pac...,"OnePlus Nord 2 5G (PAC-MAN, 256 GB)",OnePlus,"₹36,999",12 GB RAM 256 GB ROM,"OnePlus Nord 2 5G (PAC-MAN, 256 GB)",4500 mAh Battery,16.33 cm (6.43 inch) Display
3,https://www.flipkart.com/oneplus-nord-ce-2-lit...,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",OnePlus,"₹21,999",8 GB RAM 128 GB ROM,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",5000 mAh Battery,16.74 cm (6.59 inch) Display
4,https://www.flipkart.com/oneplus-nord-ce-2-5g-...,"OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)",OnePlus,"₹24,999",8 GB RAM 128 GB ROM,"OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)",4500 mAh Battery,16.33 cm (6.43 inch) Display
5,https://www.flipkart.com/oneplus-nord-2-5g-blu...,"OnePlus Nord 2 5G (Blue Haze, 128 GB)",OnePlus,"₹29,125",8 GB RAM 128 GB ROM,"OnePlus Nord 2 5G (Blue Haze, 128 GB)",4500 mAh Battery,16.33 cm (6.43 inch) Display
6,https://www.flipkart.com/oneplus-nord-gray-ony...,"OnePlus Nord (Gray Onyx, 64 GB)",OnePlus,"₹24,999",6 GB RAM 64 GB ROM,"OnePlus Nord (Gray Onyx, 64 GB)",4115 mAh Battery,16.36 cm (6.44 inch) Full HD+ Display
7,https://www.flipkart.com/oneplus-nord-gray-ony...,"OnePlus Nord (Gray Onyx, 256 GB)",OnePlus,"₹29,999",12 GB RAM 256 GB ROM,"OnePlus Nord (Gray Onyx, 256 GB)",4115 mAh Battery,16.36 cm (6.44 inch) Full HD+ Display
8,https://www.flipkart.com/oneplus-nord-gray-ony...,"OnePlus Nord (Gray Onyx, 128 GB)",OnePlus,"₹27,999",8 GB RAM 128 GB ROM,"OnePlus Nord (Gray Onyx, 128 GB)",4115 mAh Battery,16.36 cm (6.44 inch) Full HD+ Display
9,https://www.flipkart.com/oneplus-nord-blue-mar...,"OnePlus Nord (Blue Marble, 128 GB)",OnePlus,"₹27,999",8 GB RAM 128 GB ROM,"OnePlus Nord (Blue Marble, 128 GB)",4115 mAh Battery,16.36 cm (6.44 inch) Full HD+ Display


In [19]:
#Saving data to csv file
Mobiles.to_csv('Mobiles.csv')

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [20]:
import re

driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.google.com/maps/")
time.sleep(3)
city=input("Enter the name of address :")     #input field to enter location
loc=driver.find_element_by_id("searchboxinput")  #send keys to the search field 
loc.send_keys(city)
time.sleep(3)
search=driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
search.click()                    #Click on search button
time.sleep(3)

try:
    url_string=driver.current_url            #To get current url of location
    print("url_extracted :",url_string)
    lat_long=re.findall(r'@(.*)data',url_string)  #Extracting url 
    if len(lat_long):
        lat_long_list=lat_long[0].split(',')
        if len(lat_long_list)>=2:
            lat=lat_long_list[0]                   #Scraping latitude detail of url
            long=lat_long_list[1]                  #Scraping longitude detail of url
        print("Latitude = {}, Longitude = {}".format(lat, long)) 

except Exception as e:
    print("error :",str(e))
            

Enter the name of address :vadodara
url_extracted : https://www.google.com/maps/place/Vadodara,+Gujarat/@22.3223601,73.0329963,11z/data=!3m1!4b1!4m5!3m4!1s0x395fc8ab91a3ddab:0xac39d3bfe1473fb8!8m2!3d22.3071588!4d73.1812187
Latitude = 22.3223601, Longitude = 73.0329963


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [21]:
#Webscraping of trak.in platform
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://trak.in/india-startup-funding-investment-2015/") 
driver.maximize_window()
time.sleep(3)

Date=[]
Startup_Name=[]
Industry=[]
Sub_vertical=[]
City=[]
Investers_Name=[]
Investment_Type=[]
Amount=[]
#Scraping funding deals of second quarter details
x=driver.find_elements_by_xpath
d=x("//td[@class='column-2']")  #Scraping date
s=x("//td[@class='column-3']")   #Scraping startup name
ind=x("//td[@class='column-4']")  #Scraping industry
s=x("//td[@class='column-5']")   #Scraping sub vertical column
c=x("//td[@class='column-6']")    #Scraping city
inv=x("//td[@class='column-7']")  #for investment name
inv_t=x("//td[@class='column-8']") #for investment type
a=x("//td[@class='column-9']")    #for amount
for i in d:
    Date.append(str(i.text).replace('\n',''))
for i in s:
    Startup_Name.append(str(i.text).replace('\n',''))
for i in ind:
    Industry.append(str(i.text).replace('\n',''))
for i in s:
    Sub_vertical.append(str(i.text).replace('\n',''))
for i in c:
    City.append(str(i.text).replace('\n',''))
for i in inv:
    Investers_Name.append(str(i.text).replace('\n',''))
for i in inv_t:
    Investment_Type.append(str(i.text).replace('\n',''))
for i in a:
    Amount.append(str(i.text).replace('\n',''))
time.sleep(3)

print(len(Date),len(Startup_Name),len(Industry),len(Sub_vertical),len(City),len(Investers_Name),len(Investment_Type),len(Amount))

#Making dataframe
Trak=pd.DataFrame([])
Trak['Date']=Date[5:29]
Trak['Startup Name']=Startup_Name[5:29]
Trak['Industry']=Industry[5:29]
Trak['Sub Vertical']=Sub_vertical[5:29]
Trak['City']=City[5:29]
Trak['Investers Name']=Investers_Name[5:29]
Trak['Investment Type']=Investment_Type[5:29]
Trak['Amount(in USD)']=Amount[5:29]

Trak

392 392 392 392 392 392 392 392


,Date,Startup Name,Industry,Sub Vertical,City,Investers Name,Investment Type,Amount(in USD)
0,04/03/2021,Online shopping platform,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,31/03/2021,Conversational Service Automation (CSA),Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,30/03/2021,Hyper-local delivery app,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,30/03/2021,Online tutoring,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,23/03/2021,Career and job-oriented upskilling.,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,25/03/2021,Social ecommerce and online grocery platform,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,26/03/2021,Commerce and payments platform to offline ente...,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"
7,11/02/2021,E-Learning Platform,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
8,22/02/2021,Online Food Delivery Platform,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
9,19/02/2021,Semi-cooked food delivery app,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [22]:
#Webscraping of digit.in website using chrome drive
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.digit.in/")  
driver.maximize_window()   #maximize window
time.sleep(3)

search_icon=driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[3]/a") #click on laptop 
search_icon.click()

search_field=driver.find_element_by_xpath("/html/body/div[6]/div/div[2]/div[2]/ul/li[10]/a")  #click on best gaming laptops in india
search_field.click()
time.sleep(3)

brand=[]
spec=[]
dec=[]
seller=[]
price=[]

x=driver.find_elements_by_xpath
br=x("//div[@class='TopNumbeHeading active sticky-footer']")  #Scraping brand name
sp=x("//div[@class='Specs-Wrap']")            #Scraping specification details
d=x("//div[@class='Section-center']")         #Scraping laptop description
s=x("//td[@class='smmerchant']")              #Scraping laptop seller 
p=x("//td[@class='smprice']")                 #Scraping laptop price

for i in br:brand.append(str(i.text).replace("\n",""))
for i in sp:spec.append(str(i.text).replace("\n",""))
for i in d:dec.append(str(i.text).replace('\n',''))
for i in s:seller.append(str(i.text).replace('\n',''))
for i in p:price.append(str(i.text).replace('\n',''))
    
top_lap=pd.DataFrame([])                  #Makiing dataframe
top_lap['Brands']=brand[0:10]
top_lap['Specification']=spec[0:10]
top_lap['Description']=dec[0:10]
top_lap['seller']=seller[0:10]
top_lap['Price']=price[0:10]
top_lap

,Brands,Specification,Description,seller,Price
0,1.MSI RAIDER GE76,"WINDOWS 11 HOME OS17"" (3840 X 2160) DISPLAY12T...",Possibly the most high-end gaming laptop anyon...,Amazon,"₹ 429,940"
1,2.ASUS ROG STRIX SCAR 15,"WINDOWS 11 HOME OS15.6"" (2560 X 1440) DISPLAY1...",The second high-end gaming laptop on the list ...,Amazon,"₹ 280,990"
2,3.ACER NITRO 5,"WINDOWS 10 OS15.6"" (1920 X 1080) DISPLAYAMD RY...",Possibly the best value-for-money gaming lapto...,N/A,"₹ 129,990"
3,4.MSI STEALTH 15M,"WINDOWS 10 OS15.6"" (1920 X 1080) DISPLAYINTEL ...",If you’re looking for a powerful gaming laptop...,N/A,"₹ 134,990"
4,5.ASUS ROG STRIX SCAR 15,"WINDOWS 10 OS15.6"" (2560 X 1440) DISPLAYAMD RY...",If you want possibly the best performance poss...,Croma,"₹ 193,990"
5,6.ASUS ROG STRIX SCAR 15,"WINDOWS 10 HOME OS15.6"" (1920 X 1080) DISPLAYA...",When the ASUS ROG Strix Scar 15 ended up on ou...,N/A,N/A
6,7.ASUS ZEPHYRUS G14,"WINDOWS 10 HOME OS14"" (1920 X 1080) DISPLAYAMD...",The Asus Zephyrus G14 is a first-of-its-kind g...,Croma,"₹ 144,990"
7,8.HP OMEN 16,"WINDOWS 11 HOME OS16.1"" (1920 X 1080) DISPLAY1...",It is difficult to find a laptop that offers a...,Amazon,"₹ 145,500"
8,9.ASUS ROG ZEPHYRUS DUO 15,"WINDOWS 10 OS15.6"" (3840 X 1100) DISPLAYINTEL ...",The machine is powered by an Intel Core i7-108...,Amazon,"₹ 185,000"
9,10.ACER ASPIRE 7 GAMING LAPTOP,"WINDOWS 10 HOME OS15.6"" (1920 X 1080) DISPLAYA...",The Acer Aspire 7 gaming laptop is a very capa...,Croma,"₹ 53,490"


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.forbes.com/")
from selenium.webdriver.common.action_chains import ActionChains  #import mouse action library
driver.maximize_window()     #moximize window

menu=driver.find_element_by_xpath("//button[@class='icon--hamburger']")  #Click on menu list
menu.click()
time.sleep(5)

bill=driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/a") #mouse hover on billionair text
actions = ActionChains(driver)
time.sleep(3)
actions.move_to_element(bill)
time.sleep(3)
actions = ActionChains(driver)
actions.perform()
time.sleep(3)

world=driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
actions = ActionChains(driver)
time.sleep(3)
actions.move_to_element(world)     #Mouse hover on world billionair text
time.sleep(3)
actions.perform()
world.click()                    #click on world billionair link text
time.sleep(3)

In [ ]:
#Scraping details of world billionair details
Rank=[]
Name=[]
Networth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]
x=driver.find_elements_by_xpath
rank=x("//div[@class='rank']")    #Scraping rank
for i in rank:
    Rank.append(i.text)
name=x("//div[@class='personName']") #Scraping name
for i in name:
    Name.append(i.text)
net=x("//div[@class='netWorth']")   #scraping networth
for i in net:
    Networth.append(str(i.text).replace('\n',''))
age=x("//div[@class='age']")       #Scraping age
for i in age:
    Age.append(i.text)
citi=x("//div[@class='countryOfCitizenship']") #Scraping city
for i in citi:
    Citizenship.append(str(i.text).replace('\n',''))
source=x("//span[@class='source-text']")    #Scraping source
for i in source:
    Source.append(str(i.text).replace('\n',''))
industry=x("//div[@class='category']")        #Scraping industry
for i in industry:
    Industry.append(str(i.text).replace('\n',''))
time.sleep(3)
print(len(Rank),len(Name),len(Networth),len(Age),len(Citizenship),len(Source),len(Industry))

#Making dataframe
Billionair=pd.DataFrame({"Rank":Rank,"Name":Name,"Networth":Networth,"Age":Age,"Citizenship":Citizenship,"Source":Source,"Industry":Industry})
Billionair

# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [ ]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.youtube.com/results?search_query=most+viewed+kannada+song+on+youtube") #youtube vidio search for most viewd kannada songs
try:
    driver.find_element_by_xpath("/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[7]/div[1]/div/div[1]/div/h3/a/yt-formatted-string").click()
    driver.maximize_window()
    
    print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 600 in order to generate more images on the website
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,600)")   
    print("End of scrolling...")
    comments=[]
    Time_posted=[]
    comment_upvote=[]
    
    comm=driver.find_elements_by_xpath("//yt-formatted-string[@class='style-scope ytd-comment-renderer']") #Scraping comments
    for i in comm:
        comments.append(str(i.text).replace('\n',''))
    time=driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text style-scope ytd-comment-renderer']") #Scraping post time
    for i in time:    
        Time_posted.append(str(i.text).replace('\n',''))
    vote=driver.find_elements_by_xpath("//ytd-comment-action-buttons-renderer[@class='style-scope ytd-comment-renderer']")  #Scraping comments upvote
    for i in vote:    
        comment_upvote.append(str(i.text).replace('\n',' '))    
    print(len(comments),len(Time_posted),len(comment_upvote))
    print(comments,Time_posted,comment_upvote)
except Exception as e:
    print("Error :",e)

In [ ]:
#Making dataframe for 500 comment details
Youtube=pd.DataFrame([])
Youtube["Comments"]=comments[0:500]
Youtube["Posted Time"]=Time_posted[0:500]
Youtube["Comment UpVotes"]=comment_upvote[0:500]
Youtube

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.hostelworld.com/")
time.sleep(3)
search_field=driver.find_element_by_xpath("//input[@class='location-text']")
search_field.click()
time.sleep(3)
search_field=driver.find_element_by_xpath("//input[@class='location-text']")
search_field.send_keys("London")
time.sleep(3)
london=driver.find_element_by_xpath("//div[@class='label']")
actions=ActionChains(driver)
actions.move_to_element(london).click().perform()
time.sleep(3)
search_go=driver.find_element_by_xpath("//div[@class='search-button']")
search_go.click()
time.sleep(3)

In [ ]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2022-07-08&to=2022-07-11&guests=2&HostelNumber=&page=1")
time.sleep(3)
Hostel_Name=[]
Distance_from_city=[]
Rating=[]
Total_review=[]
Private_price=[]
Dorm_Prices=[]


name=driver.find_elements_by_xpath("//h2[@class='title title-6']")
dist=driver.find_elements_by_xpath("//span[@class='description']")
rat=driver.find_elements_by_xpath("//div[@class='score orange big']")
tot_re=driver.find_elements_by_xpath("//div[@class='reviews']")
pr_pr=driver.find_elements_by_xpath("//a[@class='prices']//div[1]//div")
dorm_pr=driver.find_elements_by_xpath("//a[@class='prices']//div[2]")

for i in name:
    Hostel_Name.append(i.text)
for i in dist:
    Distance_from_city.append(i.text)
for i in rat:
    if i.text is None:
        Rating.append("--")
    else:
        Rating.append(i.text)
for i in tot_re:
    Total_review.append(i.text)
for i in pr_pr:
    if i.text is None:
        Private_price.append("--")
    else:
        Private_price.append(i.text)
for i in dorm_pr:
    Dorm_Prices.append(i.text)
    
Hostel=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance from city":Distance_from_city,"Total Review":Total_review,"Dorm prices":Dorm_Prices})

    
print(len(Private_price),len(Rating))
Hostel

In [ ]:
print(Rating)

In [ ]:
print(Private_price)